In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import bs4
pd.set_option('display.max.colwidth', 100)
 

#원하는 스펙, 브랜드, 페이지제한, 키워드를 전역변수로 설정
GV_TARGETSPEC = ['센서', '측정', '체크', '보행수', '심전도']
GV_TARGETBRAND = ['삼성전자', '샤오미', '화웨이']
GV_TARGEPAGE = 2 #페이지수가 너무 많아서;; 시간이 오래걸리기때문에 타겟 설정
keyword = "스마트워치"
keywordtype = 1 #검색시 정확한 결과가 나오지않는경우 type : 1, 정확한결과가 바로 나오는 경우 type : 0

# 드라이버 위치 설정
driver_loc = "c:/chromedriver/chromedriver.exe"

# 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
# options.add_argument("headless")
# options.add_argument("disable-gpu")

# 웹 드라이버 정의
driver = webdriver.Chrome(driver_loc, options=options)

#특수문자가 포함되어있는 문자열 제거 함수
# def cleanText(strdata):
#     if '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]' in strdata :
#         return
#     else :
#         pass

#정확히 keyword가 텍스트에 있는 상품만 검열
def exactpageSearch(lst) :
    if(keywordtype == 1):
        resultList = []
        for i in range(0, len(lst)) :
            if keyword in lst[i].text :
                resultList.append(lst[i])
            else :
                pass
        return resultList
    elif(keywordtype == 0) :
        return lst
    
#원하는 브랜드명으로 다시한번 찾기
def BrandSearch(lst) :
    if(GV_TARGETBRAND != []):#원하는 브랜드가 특정되지 않았을 경우 LST를 바로 반환하기 위한 if문
        resultList = []
        local_targetBrand = GV_TARGETBRAND
        for j in local_targetBrand :
            for i in range (0, len(lst)) :
                if j in lst[i].text :
                    resultList.append(lst[i])
                else :
                    pass
#         resultList = list(set(resultList))
        return resultList
    else :
        return lst

dna_url = "http://www.danawa.com/"
driver.get(dna_url)
time.sleep(1)
path0 = '//*[@id="AKCSearch"]' #검색창
searchbox = driver.find_element_by_xpath(path0)
searchbox.send_keys(keyword)
searchbox.send_keys(Keys.ENTER)
time.sleep(3)

page2path = '//*[@id="productListArea"]/div[4]/div/div/a[2]'#2페이지로 넘어가기
page2btn = driver.find_element_by_xpath(page2path)
page2btn.click()
time.sleep(5)

spliturl = driver.current_url.split("page=2")
local_targetpage = GV_TARGEPAGE
nameList = [] #상품이름을 담을 리스트
review_num_List = [] #평점갯수를 담을 리스트
pointList = [] #평점을 담을 리스트
priceList = []#가격정보를 담을 리스트
conList = []#스펙정보를 담을 리스트
specnumList = [] #스펙갯수를 담을 리스트
linkList = []#링크정보를 담을 리스트
for i in range(1, local_targetpage+1) :
    pageurl = spliturl[0]+"page="+str(i)+spliturl[1]
    driver.get(pageurl)
    time.sleep(5)
    html = driver.page_source
    bs = bs4.BeautifulSoup(html, "html.parser")
    if(len(bs.findAll("div", {"id":"nosearchArea"}))>0) :#마지막페이지의 태그정보로 for문 종료 조건 설정
        break
    else :
        prod_info = bs.findAll("div", {"class":"prod_main_info"})
        prod_info = exactpageSearch(prod_info)
        prod_info = BrandSearch(prod_info)
       
        replaceusing = 0
        for i in range(0, len(prod_info)):
        #   평점 정보 담기
            try :
                replaceusing = prod_info[i].find("div", {"class":"point_num"}).text
            except :
                replaceusing = '평점없음' #평점이 없는경우 "평점없음"을 리스트에 담기
            pointList.append(replaceusing.replace("\t","").replace("\n","").replace("점",""))
            replaceusing=0
 

        #   리뷰갯수 정보 담기
            try :    
                replaceusing = prod_info[i].find("a", {"class":"click_log_prod_content_count"}).find("strong").text
            except :
                replaceusing = "리뷰없음" #리뷰가 없는경우 "리뷰없음"을 리스트에 담기
            review_num_List.append(replaceusing.replace("\t","").replace("\n","").replace("점",""))
            replaceusing=0

        #   제품이름 정보 담기
            replaceusing = prod_info[i].find("p", {"class":"prod_name"}).find("a").text
            nameList.append(replaceusing.replace("\t","").replace("\n","").replace("점",""))
            replaceusing=0
        
        #   링크 정보 담기    
            linkList.append(prod_info[i].find("p", {"class":"prod_name"}).find("a").attrs["href"])
       
        temppriceList = []
        for i in range(0, len(prod_info)):
            for j in prod_info[i].findAll("li"):
                temppriceList.append(j.find("p", {"class":"price_sect"}).find("strong").text) #상품에대한 모든 가격정보를 temppriceList에 추가
            temppriceList.sort() #sort이용 오름차순 정리
            priceList.append(temppriceList[0]) #최소가격만 priceList에 추가
            temppriceList.clear()

        targetspec=GV_TARGETSPEC #원하는 스펙정보 리스트
        tempt = []
        splitList = []
        
        for k in prod_info:
            try :#splitList에 prod_info의 요소를 하나씩 담기
                splitList = k.find("div", {"class":"spec_list"}).text.split("/")
            except :
                conList.append("스펙정보없음")
                continue
            for i in range(0, len(splitList)):
                for j in targetspec:#원하는 타겟스펙이 있는 항목은 tempt리스트에 담기
                    if j in splitList[i]:
                        tempt.append(splitList[i]) #
                    else : 
                        continue
            tempt            
            specnumtempt = len(tempt)
            conList.append((" ".join(tempt)).replace("\t","")) #tempt에 담긴 리스트를 문자열로 변환해서 conList에 담기
            specnumList.append(specnumtempt)
            tempt.clear()
            splitList.clear()


resultcsv = pd.DataFrame(zip(nameList, conList, specnumList, pointList, review_num_List, priceList, linkList), columns = ["상품명", "스펙", "스펙갯수","평점", "리뷰갯수","가격","링크"])

resultcsv

resultcsv.to_csv("./crawlingresult.csv", encoding = 'ms949', index = False)